**TITLE:** MULTI-AGENT INTERVIEWING SYSTEM

**DEVELOPERS:**

------

# Setup Instructions for Jupyter Notebook

This notebook installs essential packages for working with LangChain, OpenAI, and other data handling tools. 

### Important Notes:
- **Google Colab Users**: If you are using Google Colab, ensure to install `google-colab` specific packages. 
- **GPU Configuration**: If using Google Colab, you can enable GPU for faster performance by going to:
  - **Runtime** > **Change runtime type** > **Hardware accelerator** and selecting **GPU**.
  
---



### Install General Utilities and Google Colab Packages

In [ ]:
# Install general utilities and widgets
%pip install pandas opendatasets nest_asyncio ipywebrtc ipywidgets IPython 

%pip install docx2txt

In [ ]:
# Only run this cell if using google-colab, else skip it
%pip install google-colab

### Install OpenAI, LangChain, and Related Tools
These packages are necessary for using OpenAI’s language models and LangChain's toolkit for search, document processing, and data handling.

In [ ]:
# OpenAI and related LangChain tools
%pip install openai langchain_openai

# LangChain Community Tools for search and document handling
%pip install langchain_community

# Typing extensions and Pydantic
%pip install typing_extensions pydantic

# LangGraph and experimental LangChain tools
%pip install langgraph langchain_experimental

In [ ]:
# For agent tools
%pip install pypdf wikipedia duckduckgo-search playwright

!playwright install

### Database Utilities, SQLAlchemy, and FAISS for Vector Storage

- **Database Utilities**: Install SQLAlchemy for database interactions.
- **FAISS**: Choose `faiss-cpu` for CPU environments or `faiss-gpu` if you've enabled GPU support on Colab.


In [ ]:
# Database utilities and SQLAlchemy
%pip install SQLAlchemy

# FAISS for vector storage and retrieval
%pip install faiss-cpu

## General Imports
This cell includes the essential imports needed to use LangChain, OpenAI, and other data handling tools in any Jupyter Notebook or Python environment.


In [ ]:
# General imports for data handling, display, and LangChain functionality
import os
import opendatasets as od
import nest_asyncio

from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio, display, clear_output, Image
import ipywidgets as widgets

import openai
from openai import OpenAI
from langchain_openai import ChatOpenAI

# LangChain and related tools
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool

# LangChain Agents and supporting libraries
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, trim_messages
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from pydantic import BaseModel
from typing import Annotated, Literal, Sequence, List
import functools
import operator
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import create_react_agent
from langchain_experimental.tools import PythonREPLTool
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.memory import MemorySaver


### Google Colab Specific Imports
This cell should be run only if you're using Google Colab.


In [ ]:
# Google Colab specific imports
try:
    from google.colab import output
    from google.colab import userdata
    from google.colab import files
    # for colab
    output.enable_custom_widget_manager()
except:
    pass

In [ ]:
nest_asyncio.apply()

## Set API Keys

In [ ]:
# Check if running on Google Colab
try:
    # Retrieve API key from Google Colab userdata (if stored there)
    open_ai_api_key = userdata.get('OPENAI_API_KEY')
except:
    # Not running on Google Colab; prompt for API key input or retrieve from environment variables
    open_ai_api_key = os.getenv('OPENAI_API_KEY') or input("Enter your OpenAI API key: ")

# Set the API key as an environment variable for universal access within the notebook
os.environ['OPENAI_API_KEY'] = open_ai_api_key
os.environ['USER_AGENT'] = 'myagent'

# Confirm setup
if open_ai_api_key:
    print(f"API key successfully set: {open_ai_api_key}")
else:
    print("API key not set. Please check your setup.")


# 1. Create Tools

In [ ]:
from langchain_core.tools import tool

## 1.1. Speech-to-text

This tool allows the user to record speech and converts it to a text using OpenAI Whisper model.



In [ ]:
client = OpenAI()

In [ ]:
def setup_audio_recorder():
    camera = CameraStream(constraints={'audio': True, 'video': False})
    recorder = AudioRecorder(stream=camera)
    display(recorder)
    return recorder

In [ ]:
def save_recording(recorder):
    audio_data = recorder.audio.value
    if audio_data:
        with open("recording.webm", "wb") as f:
            f.write(audio_data)
        return "recording.webm"
    else:
        print("No audio data was captured. Please try again.")
        return None

In [ ]:
def convert_to_wav(input_filename, output_filename="my_recording.wav"):
    if input_filename and os.path.exists(input_filename):
        os.system(f"ffmpeg -i {input_filename} -ac 1 -f wav {output_filename} -y -hide_banner -loglevel panic")
        if os.path.exists(output_filename):
            return output_filename
        else:
            print("Conversion failed.")
            return None
    else:
        print("Input file does not exist.")
        return None

In [ ]:
def transcribe_audio(filename):
    with open(filename, "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
    print("")
    print("Transcription:", transcription.text)
    return transcription.text

In [ ]:
def record_and_transcribe_candidate_answer():
    """Record and transcribe a candidate's answer on interviewers' questions."""
    # Set up the recorder
    recorder = setup_audio_recorder()

    # Create a save button
    print("")
    save_button = widgets.Button(description="Save Recording")

    # This dictionary will store the transcribed text
    transcription_result = {}

    # Define the callback function for the save button
    def on_save_clicked(button):
        # Save the recording
        webm_file = save_recording(recorder)
        if webm_file:
            # Convert to wav format
            wav_file = convert_to_wav(webm_file)
            if wav_file:
                # Transcribe the audio and store the result
                transcription_result['text'] = transcribe_audio(wav_file)

    save_button.on_click(on_save_clicked)
    display(save_button)

    # Return the transcription result dictionary
    return transcription_result

In [ ]:
# Todo:
# Try to do live transcription, rather than recording a file. 
# Take a look at https://gist.github.com/Vaibhavs10/a48d141534cc8d877937d421bb828d8e
# and https://github.com/VRSEN/langchain-agents-tutorial/blob/main/main.py

# FOSS alternative pipeline, that doesn't rely on OpenAI models
# Using HF free API instead 
# Something like https://github.com/nyrahealth/CrisperWhisper?tab=readme-ov-file#31-usage-with--transformers

## 1.2. Text Input

In [ ]:
def setup_text_input():
    text_input = widgets.Textarea(
        placeholder="Type your answer here...",
        description="Answer:",
        layout=widgets.Layout(width='500px', height='100px')
    )
    display(text_input)
    return text_input

In [ ]:
def submit_text_input(text_widget):
    user_text = text_widget.value
    if user_text.strip():
        print("\nInput:\n", user_text)
        return user_text
    else:
        print("No input was provided. Please type your answer and try again.")
        return None

In [ ]:
def record_and_submit_text():
    """Record a candidate's text answer on interviewers' questions which require written output like code."""
    # Set up the text input widget
    text_widget = setup_text_input()

    # Create a submit button
    print("")
    submit_button = widgets.Button(description="Save Answer")

    # This variable will store the submitted text
    submission_result = {}

    # Define the callback function for the submit button
    def on_submit_clicked(button):
        # Capture the user's text input and store it in the dictionary
        submission_result['text'] = submit_text_input(text_widget)

    submit_button.on_click(on_submit_clicked)
    display(submit_button)

    # Wait for user input to be submitted
    return submission_result

## 1.3. File Reader

File Reader for PDF and DOCX files. It can extract all the text from the files, allowing us to use the files with our LLM agents.

You can upload any files, but we have set a CV and job description.

Set the file paths:

In [ ]:
cv_file_path = r"all_agents_tutorials\data\CV.pdf"
job_description_file_path = r"all_agents_tutorials\data\job_decription.docx"

The following function, extract_ccv_text, is what we will use to extract text from files in .pdf or .docx format. It uses specialized loaders (PyPDFLoader for PDFs and Docx2txtLoader for Word documents) to handle text extraction efficiently. Below is a breakdown of the code and suggestions for improvement.

In [ ]:
def extract_ccv_text(file_path: str) -> str:
    """
    Extracts text content from a given file in .pdf or .docx format.

    Args:
        file_path (str): Path to the input file (must end with .pdf or .docx).
    
    Returns:
        str: Extracted text content from the file.
    
    Raises:
        ValueError: If the file type is unsupported (not .pdf or .docx).
        Exception: For any issues encountered during text extraction.
    
    Example:
        text = extract_ccv_text("example.pdf")
        print(text)
    """
    text = ''

    try:
        if file_path.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif file_path.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            raise ValueError(f"Unsupported file type: {file_path.split('.')[-1]}")

        # Process each page and concatenate its content
        for page in loader.load():
            text += page.page_content
    except Exception as e:
        raise Exception(f"An error occurred while extracting text: {e}")
    
    return text


In [ ]:
from pprint import pprint

pprint(extract_ccv_text(cv_file_path))

In [ ]:
pprint(extract_ccv_text(job_description_file_path))

## 1.4. Hiring Company Info Web Scraper

In [ ]:
def get_wikipedia_content(query):
    """Fetches content from Wikipedia based on a query."""
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    wikipedia_content = wikipedia.run(query)
    return wikipedia_content

In [ ]:
def get_websites_links(query):
    """Fetches a list of website links based on a search query using DuckDuckGo."""
    search = DuckDuckGoSearchResults(output_format="list")
    search_results = search.invoke(query)
    return [result["link"] for result in search_results]

In [ ]:
import requests

def load_websites_content(websites):
    """
    Loads the HTML content of a list of websites synchronously using the `requests` library.

    Args:
        websites (list): A list of website URLs to fetch content from.

    Returns:
        list: A list of HTML content from the provided websites.

    Raises:
        Exception: If fetching any website fails.
    """
    content_list = []
    for website in websites:
        try:
            response = requests.get(website)
            response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
            content_list.append(response.text)
        except requests.exceptions.RequestException as e:
            print(f"Failed to fetch {website}: {e}")
            content_list.append(None)  # Optionally add `None` for failed requests
    return content_list


In [ ]:
def transform_html_content(html_content_list, tags=["span", "p", "b", "h3", "h4"]):
    """
    Transforms HTML content to extract specific tags using BeautifulSoup.

    Args:
        html_content_list (list): A list of raw HTML strings.
        tags (list): HTML tags to extract content from.

    Returns:
        list: Extracted and cleaned content from the HTML strings.
    """
    transformed_content = []
    bs_transformer = BeautifulSoupTransformer()

    # Wrap HTML strings in Document objects
    documents = [Document(page_content=html) for html in html_content_list]

    # Transform the documents using BeautifulSoupTransformer
    docs_transformed = bs_transformer.transform_documents(documents, tags_to_extract=tags)

    # Extract and store the transformed content
    for doc in docs_transformed:
        transformed_content.append(doc.page_content)

    return transformed_content


In [ ]:
def get_web_content(query):
    """
    Main function to gather content from Wikipedia and websites based on a query.

    Args:
        query (str): Search query for fetching content.

    Returns:
        list: Combined content from Wikipedia and scraped websites.
    """
    content = []

    # Fetch content from Wikipedia
    wikipedia_content = get_wikipedia_content(query)
    if wikipedia_content:
        content.append(wikipedia_content)

    # Fetch website links based on the query
    website_links = get_websites_links(f"What is {query}?")

    # Load raw HTML content from the fetched links
    html_content_list = load_websites_content(website_links)

    # Transform and clean the HTML content
    transformed_content = transform_html_content(html_content_list)

    # Combine Wikipedia and transformed website content
    content.extend(transformed_content)
    return content


In [ ]:
company_name = "Google"
websites_content = get_web_content(query=company_name)
websites_content

This is too long, so we'll call our LLM and ask it to summarize for us

In [ ]:
llm = ChatOpenAI(model_name="gpt-4o-mini")

# Ensure websites_content is a list of strings
# Join the list into a single string with proper separator (e.g., "\n\n" for readability)
system_prompt = f"""
Summarize the text from this list:

{''.join(websites_content)}

To make an overview of the contents:
"""

response = llm([{"role": "system", "content": system_prompt}])
company_info = response.content.strip()

print(company_info)


## 1.5. Dataset Querying

This is an optional tool for enhancing the process of hard skills review.

The dataset can be changed depending on the needs of users.

In [ ]:
ds = "https://www.kaggle.com/datasets/syedmharis/software-engineering-interview-questions-dataset"

In [ ]:
def get_kaggle_ds(dataset_url):
    od.download(dataset_url)

In [ ]:
# Load CSV

# Set the file path to the downloaded data and the encoding of the file
file_path = r"C:\Users\DMA\Downloads\Software Questions.csv"
encoding = "ISO-8859-1"  # default English encoding

loader = CSVLoader(file_path=file_path, encoding=encoding)
docs = loader.load()


In [ ]:
# Define text splitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)


### 5.1a Using OpenAI Embeddings 

In [ ]:
def create_questions_dataset_retriever(texts, k):
    embeddings = OpenAIEmbeddings()
    db = FAISS.from_documents(texts, embeddings)

    retriever = db.as_retriever(search_kwargs={"k": k})

    return retriever

In [ ]:
create_questions_dataset_retriever(texts=texts, k=5)

### 5.1b Using HuggingFace Embeddings 

To represent each chunk as a high-dimensional vector, we’ll use Hugging Face's pre-trained model sentence-transformers/all-MiniLM-L6-v2. This model is efficient and well-suited for generating text embeddings.


We’ll define a simple helper class to handle embedding generation using the Hugging Face model.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

class HuggingFaceEmbeddings:
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        # Load the model and tokenizer from Hugging Face
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def embed_texts(self, texts):
        # Generate embeddings for each text
        embeddings = []
        for text in texts:
            inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
            with torch.no_grad():
                outputs = self.model(**inputs)
                embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
        return np.array(embeddings)

Now, let’s generate embeddings for each of the text chunks.

In [ ]:
# Initialize the embedding model
embeddings_model = HuggingFaceEmbeddings()

# Generate embeddings for each chunk of text
embeddings = embeddings_model.embed_texts([text.page_content for text in texts])

After this step, `embeddings` will contain a vector representation of each document chunk.

To make our embeddings searchable, we’ll use FAISS to create an index. This allows us to find the most similar embeddings to any query.

In [ ]:
import faiss

# Initialize the FAISS index
embedding_dim = embeddings.shape[1]  # Dimension of embeddings
faiss_index = faiss.IndexFlatL2(embedding_dim)

# Add the embeddings to the FAISS index
faiss_index.add(embeddings)

Finally, we’ll define a `retriever` function that, given a query, will embed it and retrieve the most similar document chunks from the FAISS index.

In [ ]:
def retriever(query, texts, embeddings_model, faiss_index, k=5):
    # Generate embedding for the query
    query_embedding = embeddings_model.embed_texts([query])[0]
    
    # Search FAISS index for the top-k similar chunks
    distances, indices = faiss_index.search(np.array([query_embedding]), k)
    
    # Retrieve the corresponding text chunks
    results = [texts[i].page_content for i in indices[0]]
    return results


For testing it:

In [ ]:
# Define your query
query = "What is the topic of interest?"

# Call the retriever with the required arguments
results = retriever(query, texts, embeddings_model, faiss_index, k=5)

# Print the top results
print("Top similar chunks:")
for i, result in enumerate(results, 1):
    print(f"{i}. {result}")


### 5.2 Define the tool for agents

In [ ]:
# todo: update this tool so its usable by agents

# @tool
questions_database_tool = create_retriever_tool(
    create_questions_dataset_retriever,
    "search_subject_matter_questions",
    "Searches and returns subject matter questions for checking hard skills.",
)

# 2. Creating the Graph(s)

## Workflow Structure

- **Orchestration Stage:**
  - **Information Gathering:**
    - Obtain user documents (CV, Job description) using the `FileUpload` tool.
    - Retrieve company information via the `WebScraper` tool.
    - Create a "Scene" to outline the interview structure.

- **Interview Stages:**
  - **HR Stage:** Introduce the user, ask general and behavioral questions, and evaluate responses.
  - **Manager Stage:** Introduce the team, ask role-specific questions, and evaluate responses.
  - **Technical/Field Expert Stage:** Pose field-specific questions, assess technical knowledge, and evaluate responses.

- **Feedback Stage:** Compile evaluations from all stages and generate final feedback for the user.

Imports

In [ ]:
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool

Initialize models

In [ ]:
llm = ChatOpenAI(model_name="gpt-4o-mini")

## 2.1. Define Orchestration State

In [ ]:
user_company_name = input(prompt="What is the name of the company you are applying to?")

In [ ]:
class OrchestrationState(BaseModel):
    company_name:str = user_company_name
    user_documents: dict = Field(default_factory=dict)
    company_info: str = ""
    scene: str = ""

Define the tools for the Orchestration stage

In [ ]:
@tool
def FileUpload(description: str) -> dict:
    """
    Simulate a file upload process.

    Args:
        description (str): A brief description of the file upload context.

    Returns:
        dict: A dictionary containing simulated content for 'CV' and 'Job Description'.
    """

    ccv_text = extract_ccv_text(cv_file_path)
    job_description = extract_ccv_text(job_description_file_path)
    
    return {"CV": ccv_text, "Job Description": job_description}

@tool
def WebScraper(company_name: str) -> str:
    """
    Simulate web scraping to extract company information.

    Args:
        company_name (str): The name of a company

    Returns:
        str: Simulated company information extracted from the website.
    """
    web_content = get_web_content(company_name)
    content = "\n".join(web_content)

    return content


Define Nodes for the three sub-stages of the Orchestration stage:
- Document upload
- Scrape information about company
- Create a "Scene" to outline the interview structure.


In [ ]:
GENERATE_SCENE_SYSTEM_PROMPT = """
    Based on the following inputs:
    - CV: {cv}
    - Job Description: {job_description}
    - Company Info: {company_info}

    Generate a detailed "scene" for conducting an interview process. 
    The interview will consist of three distinct parts:
    1. **HR Screening**: Focus on cultural fit, general background, and the candidate's interest in the role and company.
    2. **Managerial Round**: Evaluate leadership skills, team collaboration, and alignment with the role’s strategic objectives.
    3. **Technical Assessment**: Deep dive into the candidate’s technical knowledge, problem-solving skills, and expertise relevant to the position.

    The scene should encourage creativity while providing a clear structure for each round, offering general guidance on what topics or approaches the participants (HR, Manager, and Technical Interviewer) might adopt.

    Your response should include:
    - A brief narrative for each part of the interview.
    - Example questions or discussion points for each section.
    - Notes on the overall flow or tone of the interview.
"""


In [ ]:
# File Upload Node
# Using the FileUpload tool we defined before
def upload_documents(state: OrchestrationState):
    state.user_documents = FileUpload("Upload user documents (CV, job description).")
    return state

# Web Scraper Node
# Using the WebScraper tool we defined before
def scrape_company_info(state: OrchestrationState):
    state.company_info = WebScraper(state.company_name)  # we can pass along the company name directly from the State
    return state

# Scene Generation Node
# Grabbing all the necessary info directly from the State
def generate_scene(state: OrchestrationState):
    cv = state.user_documents.get("CV", "")
    job_description = state.user_documents.get("Job Description", "")
    company_info = state.company_info

    system_prompt = GENERATE_SCENE_SYSTEM_PROMPT
    response = llm([{"role": "system", "content": system_prompt}])
    state.scene = response.content.strip()
    return state

Create the graph for the orchestration stage

In [ ]:
# Create the graph for orchestration
orchestration_graph = StateGraph(OrchestrationState)

# Add the nodes for each of the "sub-stages"
orchestration_graph.add_node('upload_documents', upload_documents)
orchestration_graph.add_node('scrape_company_info', scrape_company_info)
orchestration_graph.add_node('generate_scene', generate_scene)

# Define the flow of tasks
orchestration_graph.add_edge(START, 'upload_documents')
orchestration_graph.add_edge('upload_documents', 'scrape_company_info')
orchestration_graph.add_edge('scrape_company_info', 'generate_scene')
orchestration_graph.add_edge('generate_scene', END)

# Compile the orchestration graph
orchestration_app = orchestration_graph.compile()

Visualize orchestration graph

In [ ]:
display(Image(orchestration_app.get_graph().draw_mermaid_png()))

In [ ]:
initial_orchestration_state = OrchestrationState()
result_orchestration_state = orchestration_app.invoke(initial_orchestration_state)
result_orchestration_state = OrchestrationState(**result_orchestration_state)
print(type(result_orchestration_state))
print(result_orchestration_state.scene)


## 2.2. Interview Stage

In [ ]:
class InterviewState(BaseModel):
    scene: str = ""
    user_documents: dict = {}
    company_info: str = ""
    questions: list = []
    user_responses: dict = {}
    evaluation: str = ""

Define the tools for the Interview stage

In [ ]:
@tool
def UserInput(prompt: str) -> str:
    """
    Prompt the user for input.

    Args:
        prompt (str): The message displayed to the user.

    Returns:
        str: The user's input as a string.
    """

    print(prompt)
    voice, text_input = display_input_form_with_return()
    answer = f"Answer: {voice.get('text', '') if voice else ''}\n\n{text_input.get('text', '') if text_input else ''}"


    return input(prompt)


There will be three sub-stages within the Interview stage: 
- The HR interview
- The Manager interview
- The technical interview

These are quite similar, so they share the base State (InterviewState) and the tools. 

### 2.2.2. HR Interview sub-stage

For the HR Interview sub-stage, there are going to be the following tasks, which we will define as nodes for our graph: 
1. Introduction: Introduces the candidate and sets the stage for the interview.
2. Generate Questions Node: Uses the `Scene`, `User Documents`, and `Company Information` to generate a list of interview questions.
3. Ask Questions Node: Iterates through the generated questions, asks the user, and records responses.
4. Write Evaluation Node: Summarizes the candidate’s responses and writes an evaluation.

#### Nodes

In [ ]:
# Introduction
def introduction(state: InterviewState):
    print(f"Welcome to the interview! Here's an overview: {state.scene}")
    return state

In [ ]:
# Generating questions

def generate_questions(state: InterviewState):
    system_prompt = f"""
    You are an HR assistant conducting an interview.
    Use the following information to generate between 1 and 3 tailored questions:

    Scene: {state.scene}
    CV: {state.cv}
    Job Description: {state.job_description}
    Company Information: {state.company_info}
"""
    response = llm([{"role": "system", "content": system_prompt}])
    state.questions = response.content.strip().split('\n')[:3]  # list slicing to cut off at max 3 questions
    return state

In [ ]:
# Ask questions, using the UserInput tool

def ask_questions(state: InterviewState):
    while state.questions:
        question = state.questions.pop(0)
        response = UserInput(question)
        state.user_responses[question] = response
    return state

In [ ]:
# Writing the final evaluation

def write_evaluation(state: InterviewState):
    system_prompt = f"""
    Based on the following user responses, write a brief evaluation of the candidate:

    Responses: {state.user_responses}
    """
    response = llm([{"role": "system", "content": system_prompt}])
    state.evaluation = response.content.strip()
    print("Evaluation:", state.evaluation)
    return state


#### Subgraph

In [ ]:
# Create the HR sub-graph
hr_graph = StateGraph(InterviewState)

# Add nodes
hr_graph.add_node('introduction', introduction)
hr_graph.add_node('generate_questions', generate_questions)
hr_graph.add_node('ask_questions', ask_questions)
hr_graph.add_node('write_evaluation', write_evaluation)

# Define edges
hr_graph.add_edge(START, 'introduction')
hr_graph.add_edge('introduction', 'generate_questions')
hr_graph.add_edge('generate_questions', 'ask_questions')
hr_graph.add_edge('ask_questions', 'write_evaluation')
hr_graph.add_edge('write_evaluation', END)

# Compile the HR graph
hr_app = hr_graph.compile()

In [ ]:
# visualize graph

display(Image(hr_app.get_graph().draw_mermaid_png()))

### 2.2.3. Manager Interview Sub-Stage

The Manager interview sub-stage is going to be very similar to the HR interview in its structure, but it will differ in the System Prompts we give to the Manager agent, in order to gear it more towards organizational and technical questions. 

#### Nodes

In [ ]:
# Introduction
def introduction(state: InterviewState):
    print(f"Welcome to the interview! Here's an overview: {state.scene}")
    return state

In [ ]:
# Generating questions

def generate_questions(state: InterviewState):
    system_prompt = f"""
    You are an HR assistant conducting an interview.
    Use the following information to generate between 1 and 3 tailored questions:

    Scene: {state.scene}
    CV: {state.cv}
    Job Description: {state.job_description}
    Company Information: {state.company_info}
"""
    response = llm([{"role": "system", "content": system_prompt}])
    state.questions = response.content.strip().split('\n')[:3]  # list slicing to cut off at max 3 questions
    return state

In [ ]:
# Ask questions, using the UserInput tool

def ask_questions(state: InterviewState):
    while state.questions:
        question = state.questions.pop(0)
        response = UserInput(question)
        state.user_responses[question] = response
    return state

In [ ]:
# Writing the final evaluation

def write_evaluation(state: InterviewState):
    system_prompt = f"""
    Based on the following user responses, write a brief evaluation of the candidate:

    Responses: {state.user_responses}
    """
    response = llm([{"role": "system", "content": system_prompt}])
    state.evaluation = response.content.strip()
    print("Evaluation:", state.evaluation)
    return state


#### Subgraph

In [ ]:
# Create the Manager sub-graph
manager_graph = StateGraph(InterviewState)

# Add nodes
manager_graph.add_node('introduction', introduction)
manager_graph.add_node('generate_questions', generate_questions)
manager_graph.add_node('ask_questions', ask_questions)
manager_graph.add_node('write_evaluation', write_evaluation)

# Define edges
manager_graph.add_edge(START, 'introduction')
manager_graph.add_edge('introduction', 'generate_questions')
manager_graph.add_edge('generate_questions', 'ask_questions')
manager_graph.add_edge('ask_questions', 'write_evaluation')
manager_graph.add_edge('write_evaluation', END)

# Compile the Manager graph
manager_app = manager_graph.compile()

In [ ]:
# visualize graph

display(Image(manager_app.get_graph().draw_mermaid_png()))

### 2.2.4. Field-Specific Interview

The field-specific, or technical interiew, is geared towards asking questions about specifics in the field. For our example, the Field Expert agent can ask programming questions related to Python, Data Science libraries mentioned in the Job Description, etc. 

Due to this, this interview will differ slightly in its structure, in that there will be no introductory phase, and the questions can be pulled from a database of questions, if one has been provided.

In [ ]:
from pydantic import BaseModel

class ExpertInterviewState(BaseModel):
    scene: str = ""
    user_documents: dict = {}
    company_info: str = ""
    generated_query: str = ""
    retrieved_context: str = ""
    questions: list = []
    user_responses: dict = {}
    evaluation: str = ""
    db_is_available: bool = False


#### Tool 

In [ ]:
@tool
def query_questions_database(query: str) -> str: # TODO on pause do not use for now
    """
    Query the FAISS database for the most relevant questions or context.

    Args:
        query (str): The input query.

    Returns:
        str: The retrieved documents as a string.
    """
    try:
        # Retrieve relevant documents
        results = retriever.get_relevant_documents(query)

        # Format results as a string for the tool's output
        formatted_results = "\n".join([doc.page_content for doc in results])
        return formatted_results
    except Exception as e:
        return f"Error querying the database: {str(e)}"


#### Nodes

Placeholders, prompts need to be updated

In [ ]:
def generate_query(state: InterviewState):
    state.generated_query = f"""
    Generate questions for the following:
    - Scene: {state.scene}
    - CV: {state.user_documents.get('CV', '')}
    - Job Description: {state.user_documents.get('Job Description', '')}
    - Company Information: {state.company_info}

    Focus on retrieving relevant topics for the interview.
    """
    return state


def retrieve_context(state: InterviewState):
    if "DB_AVAILABLE" in globals():  # Check if the database is available
        # Use the generated query to search the database
        results = query_questions_database(state.generated_query)
        state.retrieved_context = "\n".join(results.split('\n'))  # Combine matches into context
    else:
        state.retrieved_context = ""  # No context retrieved if DB is unavailable
    return state


def generate_questions(state: InterviewState):
    # Generate questions using retrieved context (if available) and other information
    system_prompt = f"""
    Using the retrieved context below, generate three new interview questions:
    
    Retrieved Context:
    {state.retrieved_context}
    
    Additional Information:
    - Scene: {state.scene}
    - CV: {state.user_documents.get('CV', '')}
    - Job Description: {state.user_documents.get('Job Description', '')}
    - Company Information: {state.company_info}
    """
    response = llm([{"role": "system", "content": system_prompt}])
    state.questions = response.content.strip().split('\n')
    return state

def ask_questions(state: InterviewState):
    while state.questions:
        question = state.questions.pop(0)
        response = UserInput(question)  # Capture user input via tool
        state.user_responses[question] = response
    return state

def evaluate_answers(state: InterviewState):
    system_prompt = f"""
    Evaluate the following user responses based on the context:
    - Scene: {state.scene}
    - CV: {state.user_documents.get('CV', '')}
    - Job Description: {state.user_documents.get('Job Description', '')}
    - Company Information: {state.company_info}
    - User Responses: {state.user_responses}
    
    Provide a detailed evaluation.
    """
    response = llm([{"role": "system", "content": system_prompt}])
    state.evaluation = response.content.strip()
    return state


#### Subgraph

In [ ]:
# Create the Interview sub-graph
expert_interview_graph = StateGraph(ExpertInterviewState)

# Add nodes to the graph
expert_interview_graph.add_node('generate_query', generate_query)
expert_interview_graph.add_node('retrieve_context', retrieve_context)
expert_interview_graph.add_node('generate_questions', generate_questions)
expert_interview_graph.add_node('ask_questions', ask_questions)
expert_interview_graph.add_node('evaluate_answers', evaluate_answers)

# Define the condition function for the start node
def start_condition(state: ExpertInterviewState):
    if state.db_is_available:
        return 'generate_query'
    else:
        return 'generate_questions'

# Add edges
expert_interview_graph.add_conditional_edges(START, start_condition, {'generate_query': 'generate_query', 'generate_questions': 'generate_questions'})
expert_interview_graph.add_edge('generate_query', 'retrieve_context')
expert_interview_graph.add_edge('retrieve_context', 'generate_questions')
expert_interview_graph.add_edge('generate_questions', 'ask_questions')
expert_interview_graph.add_edge('ask_questions', 'evaluate_answers')
expert_interview_graph.add_edge('evaluate_answers', END)

# Compile the graph
expert_interview_app = expert_interview_graph.compile()


In [ ]:
# visualize graph

display(Image(expert_interview_app.get_graph().draw_mermaid_png()))

### 2.2.5. Connecting the interview subgraphs


In [ ]:
interview_graph = StateGraph(BaseModel)

# Add HR stage as a subgraph
interview_graph.add_node('hr', hr_app)

# Add Manager stage as a subgraph
interview_graph.add_node('manager', manager_app)

# Add Field expert stage as a subgraph
interview_graph.add_node('expert', expert_interview_app)


# add edges
interview_graph.add_edge(START, "hr")
interview_graph.add_edge("hr", "manager")
interview_graph.add_edge("manager", "expert")
interview_graph.add_edge("expert", END)
parent_graph = interview_graph.compile()

# Compile the graph
interview_app = interview_graph.compile()


In [ ]:
# visualize graph

mermaid_diagram = interview_app.get_graph(xray=True)


## 2.3. Feedback subgraph

This will be the simplest stage, where we pass along the context from the Orchestration stage, the feedback of each of the interviewers, and generate a comprehensive feedback document as the final summary. The final output will be saved as a PDF, using a markdown to PDF tool we will define. 

#### PDF writing tool

In [ ]:
%pip install markdown reportlab beautifulsoup4

In [ ]:
from markdown import markdown
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter

In [ ]:
@tool
def markdown_to_pdf(markdown_text: str) -> str:
    """
    Converts a Markdown string to a PDF file.

    Args:
    - markdown_text (str): The Markdown-formatted string.

    Returns:
    - str: Confirmation message upon successful creation of the PDF.
    """
    # Convert Markdown to HTML
    html_content = markdown(markdown_text)
    
    # Create a PDF document
    doc = SimpleDocTemplate("feedback.pdf", pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    # Split HTML into paragraphs
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")
    for element in soup.find_all(["p", "h1", "h2", "h3", "h4", "h5", "h6", "li"]):
        style = styles["BodyText"]
        if element.name.startswith("h"):
            style = styles["Heading{}".format(min(int(element.name[1]), 4))]
        story.append(Paragraph(element.text, style))

    # Build the PDF
    doc.build(story)
    return "PDF generated: 'feedback.pdf'"

#### State

In [ ]:
class FeedbackState(BaseModel):
    hr_feedback: str = ""
    manager_feedback: str = ""
    field_specialist_feedback: str = ""
    consolidated_feedback: str = ""


Nodes

In [ ]:
def gather_feedback(state: FeedbackState, hr_state: InterviewState, manager_state: InterviewState, expert_state: ExpertInterviewState):
    state.hr_feedback = hr_state.evaluation
    state.manager_feedback = manager_state.evaluation
    state.field_specialist_feedback = expert_state.evaluation
    return state

def write_feedback(state: FeedbackState):
    # Construct the system prompt for the LLM
    system_prompt = """
    You are an experienced interviewer assistant. Based on the feedback provided by the HR team, the Manager, and the Field Specialist, generate a professional and concise final feedback for the candidate.
    Ensure the feedback is holistic, constructive, and highlights key points.

    Here is the feedback from each stage:
    """

    # Append individual feedback to the system prompt
    system_prompt += f"""
    HR Feedback:
    {state.hr_feedback}

    Manager Feedback:
    {state.manager_feedback}

    Field Specialist Feedback:
    {state.field_specialist_feedback}

    Provide a summary that combines and synthesizes all the points.
    """

    # Call the LLM to generate the final feedback
    response = llm([{"role": "system", "content": system_prompt}])
    state.consolidated_feedback = response.content.strip()
    return state

def generate_pdf(state: FeedbackState):
    confirmation_message = markdown_to_pdf(state.consolidated_feedback)
    print(confirmation_message)
    return state


Subgraph

In [ ]:
from langgraph.graph import StateGraph, START, END

# Define the feedback graph
feedback_graph = StateGraph(FeedbackState)

# Add nodes
feedback_graph.add_node('gather_feedback', gather_feedback)
feedback_graph.add_node('write_feedback', write_feedback)
feedback_graph.add_node('generate_pdf', generate_pdf)

# Define edges
feedback_graph.add_edge(START, 'gather_feedback')
feedback_graph.add_edge('gather_feedback', 'write_feedback')
feedback_graph.add_edge('write_feedback', 'generate_pdf')
feedback_graph.add_edge('generate_pdf', END)

# Compile the feedback graph
feedback_app = feedback_graph.compile()


In [ ]:
# visualize graph

display(Image(feedback_app.get_graph().draw_mermaid_png()))

# 3. Putting it all together

## 3.1. Connecting the stages

In [ ]:
parent_graph = StateGraph(BaseModel)

# Add Orchestration stage as a subgraph
parent_graph.add_node('orchestration', orchestration_app)

# Add Interview stage as a subgraph
parent_graph.add_node('interview', interview_app)

# Add Feedback stage as a subgraph
parent_graph.add_node('feedback', feedback_app)


# add edges
parent_graph.add_edge(START, "orchestration")
parent_graph.add_edge("orchestration", "interview")
parent_graph.add_edge("interview", "feedback")
parent_graph.add_edge("feedback", END)

# Compile the graph
parent_app = parent_graph.compile()


In [ ]:
# visualize graph

display(Image(parent_app.get_graph().draw_mermaid_png()))

## 3.2. Transferring data between stage `State`s

To ensure seamless data transfer from the Orchestration graph to the Interview sub-graphs, we need to:
1. Extract relevant data from the OrchestrationState (e.g., `User Documents`, `Company Info`, `Scene`).
2. Transform it into an appropriate format for the HRState.
3. Use a transformer function in the parent graph to bridge the data between the two sub-graphs.

In [ ]:
def transfer_to_interview(orchestration_state: OrchestrationState) -> InterviewState:
    return InterviewState(
        cv=orchestration_state.user_documents.get("CV", ""),
        job_description=orchestration_state.user_documents.get("Job Description", ""),
        company_info=orchestration_state.company_info,
        scene=orchestration_state.scene
    )
